In [1]:
import Pkg

Pkg.activate(joinpath(@__DIR__, ".."))
Pkg.instantiate()

Pkg.add("ToQUBO", rev="px/indicator")
Pkg.add("DisjunctiveProgramming", rev="master")

  Activating project at `~/gits/DisjunctiveToQUBO.jl`


In [1]:
using JuMP
using ToQUBO                 #px/indicator
using DisjunctiveProgramming #master
using DisjunctiveToQUBO

In [ ]:
# Set parameters
cx = [1.5, 1]
cy = [6.1, 5.9]
α = [0.75, 0.8]
d = 3 # not specified in the problem, I think it is 3

In [ ]:
# Create the model
model = GDPModel(
    # Use ToQUBO in compilation mode
    () -> ToQUBO.Optimizer(nothing)
)

In [ ]:
# Add the variables
@variable(model, 0 <= x[1:2] <= 5)
@variable(model, y[1:2], Logical)

In [ ]:
# Add the objective
@objective(model, Min, cx' * x.^2 + cy' * binary_variable.(y))

In [ ]:
# Create the disjunction constraint
@constraint(model, x[2] <= 0, Disjunct(y[1]))
@constraint(model, x[1] <= 0, Disjunct(y[2]))

disjunction(model, y, exactly1 = true) # explcitly show the keyword to add an exactly 1 constraint 

In [ ]:
# Add the global constrainbt
@constraint(model, α'x >= d)

In [ ]:
# Solve the model
optimize!(model, method = Indicator())